In [70]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
spark = SparkSession.builder.appName("PracticeProject").enableHiveSupport().getOrCreate()
spark


In [71]:
# Customers Data
customers_data = [
(101, 'Ali', 'ali@gmail.com', 'Mumbai', '2022-05-10'),
(102, 'Neha', 'neha@yahoo.com', 'Delhi', '2023-01-15'),
(103, 'Ravi', 'ravi@hotmail.com', 'Bangalore', '2021-11-01'),
(104, 'Sneha', 'sneha@outlook.com', 'Hyderabad', '2020-07-22'),
(105, 'Amit', 'amit@gmail.com', 'Chennai', '2023-03-10'),
]
orders_data = [
(1, 101, 'Laptop', 'Electronics', 2, 50000.0, '2024-01-10'),
(2, 101, 'Mouse', 'Electronics', 1, 1200.0, '2024-01-15'),
(3, 102, 'Tablet', 'Electronics', 1, 20000.0, '2024-02-01'),
(4, 103, 'Bookshelf', 'Furniture', 1, 3500.0, '2024-02-10'),
(5, 104, 'Mixer', 'Appliances', 1, 5000.0, '2024-02-15'),
(6, 105, 'Notebook', 'Stationery', 5, 500.0, '2024-03-01'),
(7, 102, 'Phone', 'Electronics', 1, 30000.0, '2024-03-02'),
]

# Write as tables
customers_df = spark.createDataFrame(customers_data, ["CustomerID", "Name", "Email",
"City", "SignupDate"])
orders_df = spark.createDataFrame(orders_data, ["OrderID", "CustomerID", "Product",
"Category", "Quantity", "Price", "OrderDate"])
spark.sql("create database if not exists sales")

DataFrame[]

In [72]:
#1. Add a column TotalAmount = Price * Quantity to the orders_df
orders_df = orders_df.withColumn("TotalAmount", col("Price") * col("Quantity"))
orders_df.show()
#2. Filter all orders with TotalAmount > 10000 .
orders_df.filter(col("TotalAmount") > 10000).show()
#3. Standardize the City field in customers_df (e.g., lowercase).
from pyspark.sql.functions import lower
customers_df = customers_df.withColumn("City", lower(col("City")))
customers_df.show()
#4. Extract year from OrderDate and add a new column OrderYear .
orders_df = orders_df.withColumn("OrderYear", year(col("OrderDate")))
orders_df.show()
#5. Fill null values in any column of your choice with defaults.
orders_df = orders_df.fillna({"Category": "Unknown"})
orders_df.show()
#6. Use when/otherwise to categorize orders:
#<5000 : "Low"
#5000-20000 : "Medium"
#>20000 : "High"
from pyspark.sql.functions import when,col
orders_df = orders_df.withColumn("Category product", when(col("TotalAmount") < 5000, "Low").otherwise(when(col("TotalAmount") < 20000, "Medium").otherwise("High")))
orders_df.show()

+-------+----------+---------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+---------+-----------+--------+-------+----------+-----------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+---------+-----------+--------+-------+----------+-----------+

+-------+----------+-------+-----------+--------+-------+----------+-----------+
|Orde

In [73]:
customers_df.write.mode("overwrite").saveAsTable("sales.customers")
orders_df.write.mode("overwrite").saveAsTable("sales.orders")

#7. Run a SQL query to list all orders made by “Ali”.
spark.sql("select * from sales.orders where customerid = (select customerid from sales.customers where name = 'Ali')").show()

#8. Get total spending by each customer using SQL.
spark.sql("""select c.Name, sum(o.Quantity * o.Price) as TotalSpending from sales.customers c join sales.orders o on c.CustomerID = o.CustomerID group by c.Name""").show()

#9. Find out which category made the highest total revenue.
spark.sql("select category, sum(quantity * price) as totalrevenue from sales.orders group by category order by totalrevenue desc limit 1").show()

#10. Create a view customer_orders showing CustomerName, Product, TotalAmount .
spark.sql("create or replace view sales.customer_orders as select c.Name as CustomerName, o.Product, (o.Quantity * o.Price) as TotalAmount from sales.customers c join sales.orders o on c.CustomerID = o.CustomerID")

#11. Query the view for products ordered after Feb 2024.
spark.sql("select * from sales.orders where orderdate > '2024-02-01'").show()


+-------+----------+-------+-----------+--------+-------+----------+-----------+---------+----------------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|Category product|
+-------+----------+-------+-----------+--------+-------+----------+-----------+---------+----------------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|            High|
|      2|       101|  Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|             Low|
+-------+----------+-------+-----------+--------+-------+----------+-----------+---------+----------------+

+-----+-------------+
| Name|TotalSpending|
+-----+-------------+
| Ravi|       3500.0|
|Sneha|       5000.0|
| Amit|       2500.0|
| Neha|      50000.0|
|  Ali|     101200.0|
+-----+-------------+

+-----------+------------+
|   category|totalrevenue|
+-----------+------------+
|Electronics|    151200.0|
+-----------+------------+

+-------+-------

In [74]:
#12. Create a Global Temp View from customers_df , then query it using:
#SELECT * FROM global_temp.customers WHERE City = 'Mumbai';
customers_df.createOrReplaceGlobalTempView("customers")
spark.sql("select * from global_temp.customers where City = 'mumbai'").show()
#13. Save the transformed orders_df (with TotalAmount) to a Parquet file.
orders_df.write.mode("overwrite").parquet("orders.parquet")
#14. Read back the Parquet file and count how many orders are in it.
orders_df_parquet = spark.read.parquet("orders.parquet")
orders_df_parquet.count()

+----------+----+-------------+------+----------+
|CustomerID|Name|        Email|  City|SignupDate|
+----------+----+-------------+------+----------+
|       101| Ali|ali@gmail.com|mumbai|2022-05-10|
+----------+----+-------------+------+----------+



7

In [75]:
#15. Write a UDF that masks emails like:
#ali@gmail.com → a***@gmail.com .
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
def mask_email(email):
    parts = email.split('@')
    if len(parts) == 2:
        username, domain = parts
        masked_username = username[0] + '*' * (len(username) - 2) + username[-1]
        return f"{masked_username}@{domain}"
    else:
        return email
mask_email_udf = udf(mask_email, StringType())
customers_df = customers_df.withColumn("Email", mask_email_udf(col("Email")))
customers_df.show()
#16. Use concat_ws() to create a full label like:
#'Ali from Mumbai'.
from pyspark.sql.functions import concat_ws
customers_df = customers_df.withColumn("FullLabel", concat_ws(" from ", col("Name"), col("City")))
customers_df.show()
#17. Use regexp_replace() to remove special characters from product names.
from pyspark.sql.functions import regexp_replace
orders_df = orders_df.withColumn("Product", regexp_replace(col("Product"), "[^a-zA-Z0-9 ]", ""))
orders_df.show()
#18. Use to_date() and datediff() to calculate customer age in days (from SignupDate to today).
from pyspark.sql.functions import datediff, to_date, current_date
customers_df = customers_df.withColumn("AgeInDays", datediff(to_date(current_date()), to_date(col("SignupDate"))))
customers_df.show()

+----------+-----+-----------------+---------+----------+
|CustomerID| Name|            Email|     City|SignupDate|
+----------+-----+-----------------+---------+----------+
|       101|  Ali|    a*i@gmail.com|   mumbai|2022-05-10|
|       102| Neha|   n**a@yahoo.com|    delhi|2023-01-15|
|       103| Ravi| r**i@hotmail.com|bangalore|2021-11-01|
|       104|Sneha|s***a@outlook.com|hyderabad|2020-07-22|
|       105| Amit|   a**t@gmail.com|  chennai|2023-03-10|
+----------+-----+-----------------+---------+----------+

+----------+-----+-----------------+---------+----------+--------------------+
|CustomerID| Name|            Email|     City|SignupDate|           FullLabel|
+----------+-----+-----------------+---------+----------+--------------------+
|       101|  Ali|    a*i@gmail.com|   mumbai|2022-05-10|     Ali from mumbai|
|       102| Neha|   n**a@yahoo.com|    delhi|2023-01-15|     Neha from delhi|
|       103| Ravi| r**i@hotmail.com|bangalore|2021-11-01| Ravi from bangalore|
|  

In [76]:
spark.stop()